In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import glob
from datetime import datetime,date
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymongo
import csv
import json
import re
import os
import psycopg2
from urllib.parse import urlencode, urlparse, parse_qsl

# Import API key
from config import g_key
from config import hostname
from config import port_id
from config import username 
from config import database 
from config import pwd

# Configure gmaps API key
gmaps.configure(api_key=g_key)

ImportError: cannot import name 'g_key' from 'config' (/Users/Anina/Desktop/Final_Project/Interactive_Map/config.py)

# Connecting to Postgres

In [2]:
#Connecting to Postgres
conn = psycopg2.connect(
        host = hostname,
        dbname = database,
        user = username,
        password = pwd, 
        port = port_id)
print("database opened successfully")

NameError: name 'hostname' is not defined

# Download from Top 10

In [ ]:
#Downloading the data from Postgres
df = pd.read_sql('SELECT * FROM top10', conn)
df.head()

In [ ]:
# FINAL PROJECT: Import DATABASE
city_data_df = df
city_data_df.head(10)

In [ ]:
city_data_df.dtypes

In [ ]:
# FINAL PROJECT: FILTER THE DF USING LOC METHOD
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df
preferred_cities_df.head(10)

In [ ]:
# FINAL PROJECT: USE BELOW BUT ADD CURRENT AND PROJECTED COVID CASES
# hotel_df = clean_df[["City", "Country", "Current Covid Cases", "Projected Covid Cases", "Current Description", "Lat", "Lng"]].copy()
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
# hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
# hotel_df["Hotel Name"] = preferred_cities_df
hotel_df = preferred_cities_df[["country_region", "city", "province_state", "lat", "long", "confirmed_cases", "people_fully_vaccinated", "predicted_values"]].copy()
hotel_df

In [ ]:
# Add Hotel Name
hotel_df["Hotel Name"] = ""
hotel_df["Rating"] = ""
hotel_df

In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 50,
    "type": "lodging",
    "key": g_key,
    "fields": "name, formatted_address, price_level, rating"
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["lat"]
    lng = row["long"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    print(hotels)
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        hotel_df.loc[index, "Rating"] = hotels ["results"][0]["rating"]
    
    except (IndexError):
        print("Hotel not found... skipping.")

In [ ]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()
print(hotel_df)

hotel_df2 = hotel_df.dropna(subset=['hotel name'])
final_df3

In [ ]:
# FINAL PROJECT: CREATE OUTPUT FILE
# output_data_file = "ENTER NEW FILE.csv"
# 8a. Create the output File (CSV)
output_data_file = "top_10_hotel_list.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file)
hotel_df.head()

In [ ]:
# FINAL PROJECT: UPDATE TEMPLATE TO INCLUDE COVID CASES AND PROJECTED COVID CASES
# info_box_template = """
# <dl>
# <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
# <dt>City Name</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# <dt>Current Weather</dt><dd>{Current Description}</dd>
# <dt>Current Covid Cases</dt><dd>{CURRENT COVID CASES}</dd>
# <dt>Projected Covid Cases</dt><dd>{PROJECTED COVID CASES}</dd>
# <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
# </dl>
# """
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Rating</dt><dd>{Rating}</dd>
<dt>City Name</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country_region}</dd>
<dt>Confirmed Covid Cases</dt><dd>{confirmed_cases}</dd>
<dt>Projected Covid Cases</dt><dd>{predicted_values}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["lat", "long"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["lat", "long"]]
fig = gmaps.figure(center=(49.372300, -2.364400), zoom_level=1)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig